In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
from lightgbm import LGBMRegressor
from matplotlib import pyplot as plt
from scipy.stats import ttest_ind,chi2_contingency
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import optuna
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool, cv, MetricVisualizer
import os
import warnings
for dirname, _, filenames in os.walk('/ekoagro/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
rc = {
    "axes.facecolor": "#F8F8F8",
    "figure.facecolor": "#F8F8F8",
    "axes.edgecolor": "#000000",
    "grid.color": "#EBEBE7" + "30",
    "font.family": "serif",
    "axes.labelcolor": "#000000",
    "xtick.color": "#000000",
    "ytick.color": "#000000",
    "grid.alpha": 0.4,
}

sns.set(rc=rc)
palette = ['#302c36', '#037d97', '#E4591E', '#C09741',
           '#EC5B6D', '#90A6B1', '#6ca957', '#D8E3E2']

from colorama import Style, Fore
blk = Style.BRIGHT + Fore.BLACK
mgt = Style.BRIGHT + Fore.MAGENTA
red = Style.BRIGHT + Fore.RED
blu = Style.BRIGHT + Fore.BLUE
res = Style.RESET_ALL

plt.style.use('fivethirtyeight')

In [ ]:
train_data = pd.read_csv("/ekoagro/input/playground-series-s4e5/train.csv")
test_data = pd.read_csv("/ekoagro/input/playground-series-s4e5/test.csv")

In [ ]:
train_data.sample(10).style.set_properties(**{'background-color': '#f9f9f9', 'color': '#4CAF50', 'font-weight': 'bold'})

In [ ]:
test_data.sample(10).style.set_properties(**{'background-color': '#f9f9f9', 'color': '#1565C0', 'font-weight': 'bold'})

In [ ]:
train_data.describe().style.set_properties(**{'background-color': '#f9f9f9', 'color': '#4CAF50', 'font-weight': 'bold'})

In [ ]:
test_data.describe().style.set_properties(**{'background-color': '#f9f9f9', 'color': '#1565C0', 'font-weight': 'bold'})

In [ ]:
train_data.info(show_counts=True)

In [ ]:
test_data.info()

In [ ]:
sns.displot(data=train_data.isnull().melt(value_name = 'missing'),
           y = 'variable',
           hue = 'missing',multiple='fill',height=8,aspect = 1.6)
plt.axvline(0.4,color = 'r')
plt.title("Null values in train data",fontsize = 13)
plt.show()

In [ ]:
sns.displot(data=test_data.isnull().melt(value_name = 'missing'),
           y = 'variable',
           hue = 'missing',multiple='fill',height=8,aspect = 1.6)
plt.axvline(0.4,color = 'r')
plt.title("Null values in train data",fontsize = 13)
plt.show()

In [ ]:
train_data.duplicated().sum()

In [ ]:
train_data.columns

In [ ]:
# Plotting
plt.figure(figsize=(14, 6))

# Plotting histogram and KDE for 'FloodProbability' column
plt.subplot(1, 2, 1)
plt.hist(train_data['FloodProbability'], bins=30, color='skyblue', edgecolor='black')
plt.title('Distribution of FloodProbability', fontsize=14, fontweight='bold')
plt.xlabel('FloodProbability', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()

# Plotting KDE only for 'FloodProbability' column
plt.subplot(1, 2, 2)
sns.kdeplot(train_data['FloodProbability'], color='black')
plt.title('Kernel Density Estimation (KDE) for FloodProbability', fontsize=14, fontweight='bold')
plt.xlabel('FloodProbability', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()

plt.show()

In [ ]:
unique_counts = train_data.nunique()
#Threshold to distinguish continous and categorical
threshold = 12
continuous_vars = unique_counts[unique_counts > threshold].index.tolist()
#categorical_vars = unique_counts[unique_counts <= threshold].index.tolist()
if 'id' in continuous_vars:
    continuous_vars.remove('id')

In [ ]:
# Set up warnings to be ignored (optional)
warnings.filterwarnings("ignore")
pd.set_option('mode.use_inf_as_na', False)

# List of continuous variables in your dataset


# Set hue to your target column
target_column = 'FloodProbability'

for column in continuous_vars:
    fig, axes = plt.subplots(1, 2, figsize=(18, 4))  # Create subplots with 1 row and 2 columns
    
    # Plot histogram with hue
    sns.histplot(data=train_data, x=column, hue=target_column, bins=50, kde=True, ax=axes[0], palette='muted')
    axes[0].set_title(f'Histogram of {column} with {target_column} Hue')
    axes[0].set_xlabel(column)
    axes[0].set_ylabel('Count')
    axes[0].legend(title=target_column, loc='upper right')
    
    # Plot KDE plot with hue
    sns.kdeplot(data=train_data, x=column, hue=target_column, ax=axes[1], palette='muted')
    axes[1].set_title(f'KDE Plot of {column} with {target_column} Hue')
    axes[1].set_xlabel(column)
    axes[1].set_ylabel('Density')
    axes[1].legend(title=target_column, loc='upper right')
    
    plt.tight_layout()  # Adjust spacing between subplots
    plt.show()

In [ ]:
import scipy.stats as stats  
def qq_plot_with_skewness(data, quantitative_var):
    # Check if the variable is present in the DataFrame
    if quantitative_var not in data.columns:
        print(f"Error: '{quantitative_var}' not found in the DataFrame.")
        return
    
    f, ax = plt.subplots(1, 2, figsize=(18, 5.5))

    # Check for missing values
    if data[quantitative_var].isnull().any():
        print(f"Warning: '{quantitative_var}' contains missing values. Results may be affected.")

    # QQ plot
    stats.probplot(data[quantitative_var], plot=ax[0], fit=True)
    ax[0].set_title(f'QQ Plot for {quantitative_var}')

    # Skewness plot
    sns.histplot(data[quantitative_var], kde=True, ax=ax[1])
    ax[1].set_title(f'Distribution of {quantitative_var}')

    # Calculate skewness value
    skewness_value = stats.skew(data[quantitative_var])

    # Display skewness value on the plot
    ax[1].text(0.5, 0.5, f'Skewness: {skewness_value:.2f}', transform=ax[1].transAxes, 
               horizontalalignment='center', verticalalignment='center', fontsize=16, color='red')

    plt.show()
# Example usage for each continuous variable
for var in continuous_vars:
    qq_plot_with_skewness(train_data, var)

In [ ]:
# Feature Interaction
train_data['Urbanization_Deforestation_Interaction'] = train_data['Urbanization'] * train_data['Deforestation']

# Ratio Features
train_data['Urbanization_TopographyDrainage_Ratio'] = train_data['Urbanization'] / (train_data['TopographyDrainage'] + 1)  # Added 1 to avoid division by zero

# Aggregated Features (Example: Sum of selected columns)
selected_columns = ['Urbanization', 'Deforestation', 'AgriculturalPractices', 'InadequatePlanning']
train_data['Sum_Selected_Columns'] = train_data[selected_columns].sum(axis=1)

In [ ]:
# Feature Interaction
test_data['Urbanization_Deforestation_Interaction'] = test_data['Urbanization'] * test_data['Deforestation']

# Ratio Features
test_data['Urbanization_TopographyDrainage_Ratio'] = test_data['Urbanization'] / (test_data['TopographyDrainage'] + 1)  # Added 1 to avoid division by zero

# Aggregated Features (Example: Sum of selected columns)
selected_columns = ['Urbanization', 'Deforestation', 'AgriculturalPractices', 'InadequatePlanning']
test_data['Sum_Selected_Columns'] = test_data[selected_columns].sum(axis=1)

In [ ]:
train_data.sample(10).style.set_properties(**{'background-color': '#f9f9f9', 'color': '#4CAF50', 'font-weight': 'bold'})

In [ ]:
test_data.sample(10).style.set_properties(**{'background-color': '#f9f9f9', 'color': '#1565C0', 'font-weight': 'bold'})

In [ ]:
unique_counts = train_data.nunique()
#Threshold to distinguish continous and categorical
threshold = 12
continuous_vars_temp = unique_counts[unique_counts > threshold].index.tolist()
#categorical_vars = unique_counts[unique_counts <= threshold].index.tolist()
if 'id' in continuous_vars_temp:
    continuous_vars_temp.remove('id')

In [ ]:
unique_counts_test = test_data.nunique()
#Threshold to distinguish continous and categorical
threshold = 12
continuous_vars_temp_test = unique_counts_test[unique_counts_test > threshold].index.tolist()
#categorical_vars = unique_counts[unique_counts <= threshold].index.tolist()
if 'id' in continuous_vars_temp_test:
    continuous_vars_temp_test.remove('id')

In [ ]:
def plot_boxplots(data, columns, ncols=2):
    nrows = (len(columns) + ncols - 1) // ncols
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 4 * nrows))

    for i, column in enumerate(columns):
        ax = axes[i // ncols, i % ncols] if nrows > 1 else axes[i % ncols]

        if data[column].dtype == 'O':  # 'O' represents object (categorical) dtype
            sns.countplot(x=column, data=data, ax=ax)
            ax.set_title(f'Countplot for {column}')
        else:
            sns.boxplot(x=column, data=data, ax=ax)
            ax.set_title(f'Boxplot for {column}')

    plt.tight_layout()
    plt.show()

plot_boxplots(train_data, continuous_vars_temp)

In [ ]:
def remove_outliers_replace(data, columns, threshold=1.5):
    data_no_outliers = data.copy()

    for column in columns:
        Q1 = data_no_outliers[column].quantile(0.25)
        Q3 = data_no_outliers[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR

        is_outlier = (data_no_outliers[column] < lower_bound) | (data_no_outliers[column] > upper_bound)

        if data_no_outliers[column].dtype == 'O':  # Categorical column
            median_value = data_no_outliers.loc[~is_outlier, column].mode().iloc[0]
            data_no_outliers.loc[is_outlier, column] = median_value
        else:  # Numerical column
            mean_value = data_no_outliers.loc[~is_outlier, column].mean()
            data_no_outliers.loc[is_outlier, column] = mean_value

    return data_no_outliers

columns_to_remove_outliers_replace = continuous_vars_temp
train_data = remove_outliers_replace(train_data, columns_to_remove_outliers_replace)

In [ ]:
columns_to_remove_outliers_replace = continuous_vars_temp_test
test_data = remove_outliers_replace(test_data, columns_to_remove_outliers_replace)

In [ ]:
train_data.drop(columns='id',axis = 1,inplace = True)

In [ ]:
train_data.sample(10).style.set_properties(**{'background-color': '#f9f9f9', 'color': '#4CAF50', 'font-weight': 'bold'})

In [ ]:
X = train_data.drop(columns='FloodProbability', axis=1)
y = train_data['FloodProbability']

In [ ]:
merged_data = pd.concat([X, y.rename('FloodProbability')], axis=1)

# Calculate the correlation matrix
correlation_matrix = merged_data.corr()

# Plot the heatmap
plt.figure(figsize=(10, 12))
sns.heatmap(correlation_matrix[['FloodProbability']], annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation with respect to FloodProbability')
plt.show()

In [ ]:
correlation_matrix = X.corr()

# Set up the matplotlib figure with a larger size
plt.figure(figsize=(18, 14))  # Increase the width and height as needed

# Plot the correlation matrix as a heatmap with larger boxes
heatmap = sns.heatmap(correlation_matrix, annot=True, cmap='Greens', fmt=".2f", linewidths=1, square=True)

# Customize plot
plt.title('Correlation Matrix', fontsize=20)  # Increase the font size of the title
plt.xticks(fontsize=12)  # Increase the font size of x-axis labels
plt.yticks(fontsize=12)  # Increase the font size of y-axis labels

# Rotate the x-axis labels for better readability
plt.xticks(rotation=45)

# Adjust the aspect ratio to prevent distortion of cell shapes
heatmap.set_aspect('equal')

plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Sort the dataframe by VIF values in descending order
vif_data = vif_data.sort_values(by='VIF', ascending=False)

# Set a VIF threshold (e.g., VIF > 5)
vif_threshold = 5

# Identify variables with high VIF that exceed the threshold
high_vif_variables = vif_data[vif_data["VIF"] > vif_threshold]

# Create a beautiful bar plot using Seaborn, highlighting high VIF variables
plt.figure(figsize=(12, 12))
sns.set(style="whitegrid")
plot = sns.barplot(x="VIF", y="Variable", data=vif_data, palette="viridis")

# Highlight high VIF variables
for bar in plot.patches:
    if bar.get_width() > vif_threshold:
        bar.set_color('coral')

plt.xlabel('VIF')
plt.ylabel('Variable')
plt.title('Variance Inflation Factor (VIF) for Independent Variables')
plt.xticks(fontsize=12)  # Increase font size of x-axis ticks
plt.yticks(fontsize=12)  # Increase font size of y-axis ticks
plt.grid(axis='x', linestyle='--', alpha=0.6, color='gray')  # Add grid lines
plt.tight_layout()  # Adjust layout for better appearance
plt.show()

In [ ]:
# Display high VIF variables
print("Variables with high VIF (> {}):".format(vif_threshold))
high_vif_variables.style.background_gradient()

In [ ]:
target_columns = ['FloodProbability']

# Perform pairwise t-tests for each target column with all other columns
p_values = {}
for target in target_columns:
    p_values[target] = {}  # Initialize a dictionary for storing p-values for this target
    for column in train_data.columns:
        if column != target:  # Exclude the target column itself
            t_stat, p_value = ttest_ind(train_data[target], train_data[column])
            p_values[target][column] = p_value

# Convert the nested dictionary to a DataFrame for better visualization
p_values_df = pd.DataFrame(p_values)

# Plotting the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(p_values_df, cmap='viridis', annot=True, fmt=".2f")
plt.title('Pairwise t-test p-values with respect to FloodProbability')
plt.xlabel('Columns')
plt.ylabel('Target Columns')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
print("P-values for Hypothesis Testing:")
p_values_df.style.background_gradient()

In [ ]:
target_columns = ['FloodProbability']

# Perform pairwise t-tests for each target column with all other columns
significant_columns = {}
for target in target_columns:
    p_values = {}  # Initialize a dictionary for storing p-values for this target
    for column in train_data.columns:
        if column != target:  # Exclude the target column itself
            t_stat, p_value = ttest_ind(train_data[target], train_data[column])
            p_values[column] = p_value
    
    # Filter columns based on p-value threshold (e.g., 0.05)
    significant_columns[target] = [col for col, p_val in p_values.items() if p_val <= 0.05]

# Display the number of significant columns for each target column
for target, cols in significant_columns.items():
    print(f"Number of significant columns for '{target}': {len(cols)}")

In [ ]:
# Display the new DataFrame with principal components
print("DataFrame with Principal Components:")

# Visualize in 3D if there are at least 3 components
if num_components >= 3:
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter(pca_df['PC1'], pca_df['PC2'], pca_df['PC3'], c=colors, cmap='viridis', alpha=0.5)
    ax.set_title('PCA Result (3D) with Different Colors')
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.set_zlabel('Principal Component 3')
    colorbar = fig.colorbar(scatter, ax=ax, pad=0.1)
    colorbar.set_label('Color Label')  # Update with appropriate label

    plt.show()
else:
    print("Number of components is less than 3, unable to visualize in 3D.")
    
pca_df.sample(10).style.background_gradient()

In [ ]:
X_scale = train_data.drop(columns='FloodProbability', axis=1)
y_scale = train_data['FloodProbability']

In [ ]:
ss = StandardScaler()

In [ ]:
X_scale_S = ss.fit_transform(X_scale)

In [ ]:
X_scale_S

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_scale_S, y, test_size=0.2, random_state=42)

In [ ]:
best_params = {'learning_rate': 0.049769346237099124, 
               'n_estimators': 584,
               'reg_alpha': 0.678327747915515, 
               'reg_lambda': 0.6307183788894067,
               'max_depth': 14, 'subsample': 0.9433308610114127,
               'colsample_bytree': 0.3492429037129386, 
               'min_child_weight': 7.658855760564173}

In [ ]:
# optimized parameters     
xgb_best_params = {
    'n_estimators': 804,
    'learning_rate': 0.07366400082037482,
    'max_depth': 5,
    'min_child_weight': 7,
    'gamma': 0.0049952867265411136, 
    'subsample': 0.5012471120723563, 
    'colsample_bytree': 0.848562748505815,
    'lambda': 1.9547551669960237, 
    'alpha': 1.5254037937212281
}

In [ ]:
model = XGBRegressor(**best_params)
model.fit(X_scale_S, y)

In [ ]:
model_xgb = XGBRegressor(**xgb_best_params)
model_xgb.fit(X_scale_S, y)

In [ ]:
feature_importance = model.feature_importances_
feature_names = X.columns

sorted_indices = feature_importance.argsort()
sorted_importance = feature_importance[sorted_indices]
sorted_features = feature_names[sorted_indices]

plt.figure(figsize=(10, 6))
colors = plt.cm.tab20c.colors[:len(sorted_features)]  
plt.barh(sorted_features, sorted_importance, color=colors)
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('XGBoost Feature Importance')
plt.gca().invert_yaxis() 
plt.tight_layout()  
plt.show()

In [ ]:
#best_params_cat = {}

In [ ]:
#model_cat = CatBoostRegressor(**best_params_cat)
model_cat = CatBoostRegressor()
model_cat.fit(X_scale_S, y)

In [ ]:
pool = Pool(X_scale_S, y)

# Get feature importance
feature_importance = model_cat.get_feature_importance(pool, type='PredictionValuesChange', prettified=True)

# Sort feature importance values
feature_importance_sorted = feature_importance.sort_values(by='Importances', ascending=False)

# Generate a color gradient for the bars
colors = sns.color_palette("viridis", len(feature_importance_sorted))

# Plot feature importance with customized colors and black border for bars
plt.figure(figsize=(10, 6))
bars = plt.barh(feature_importance_sorted['Feature Id'], feature_importance_sorted['Importances'], color=colors, edgecolor='black')  # Add edgecolor='black' for black borders
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('CatBoost Feature Importance')
plt.gca().invert_yaxis()  # Invert y-axis to have the most important features at the top
plt.tight_layout()  # Adjust layout to prevent cropping

# Add color bar legend
sm = plt.cm.ScalarMappable(cmap='viridis')
sm.set_array(feature_importance_sorted['Importances'])
cbar = plt.colorbar(sm, orientation='vertical')
cbar.set_label('Importance Color Gradient')

plt.show()

In [ ]:
#print('Best Parameters:', study.best_params)
best_params_lgbm = {
    'n_estimators': 939,
    'learning_rate': 0.2686031022851845,
    'max_depth': 3, 
    'min_child_samples': 55, 
    'num_leaves': 68,
    'subsample': 0.698948956171567,
    'colsample_bytree': 0.5834834747393846, 
    'reg_lambda': 1.0666821210651256,
    'reg_alpha': 1.9332040363807412
}

In [ ]:
import warnings
# Filter out LightGBM warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
model_lgbm = LGBMRegressor(**best_params_lgbm)
model_lgbm.fit(X_scale_S,y)

In [ ]:
feature_importance = model_lgbm.feature_importances_

feature_names = X.columns

sorted_indices = feature_importance.argsort()
sorted_importance = feature_importance[sorted_indices]
sorted_features = feature_names[sorted_indices]

# Plot feature importance
plt.figure(figsize=(12, 8))
colors = plt.cm.Paired.colors[:len(sorted_features)]  
plt.barh(sorted_features, sorted_importance, color=colors)
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('LightGBM Feature Importance', fontsize=14)
plt.gca().invert_yaxis() 

for i, v in enumerate(sorted_importance):
    plt.text(v + 0.02, i, f'{v:.2f}', color='black', va='center', fontsize=10)

plt.tight_layout()  
plt.show()

In [ ]:
test_pred = test_data.drop(columns='id',axis = 1)

In [ ]:
predictions_xg = model.predict(test_pred)

In [ ]:
predictions_xg

In [ ]:
predictions_xg2 = model_xgb.predict(test_pred)

In [ ]:
predictions_xg2

In [ ]:
prediction_catboost = model_cat.predict(test_pred)

In [ ]:
prediction_catboost

In [ ]:
prediction_lgbm = model_lgbm.predict(test_pred)

In [ ]:
prediction_lgbm

In [ ]:
submitted_xgboost = pd.read_csv("/ekoagro/input/xgboostsubmission2/XGBoost_Submission2.csv")

In [ ]:
submitted_xgboost['FloodProbability'].head(3)

In [ ]:
sample_submission = pd.read_csv("/ekoagro/input/playground-series-s4e5/sample_submission.csv")

In [ ]:
ample_submission['FloodProbability'] = predictions_xg

In [ ]:
sample_submission.to_csv("XGBoost_Submission.csv",index=False)

In [ ]:
sample_submission['FloodProbability'] = submitted_xgboost['FloodProbability']

In [ ]:
sample_submission.to_csv("XGBoost_Submission2.csv",index=False)

In [ ]:
sample_submission['FloodProbability'] = prediction_catboost

In [ ]:
sample_submission.to_csv("CatBoost_Submission.csv",index=False)

In [ ]:
sample_submission['FloodProbability'] = prediction_lgbm

In [ ]:
sample_submission.to_csv("LGMB_Submission.csv",index=False)